# Participantes do grupo  



## André Gomes Monteiro - 89168

## Leonardo Aranha - 86919 

## Luara Maria Marino - 89375 

## Renato Kenji Yamashiro - 88847

## Instalação dos módulos para as bibliotecas

In [1]:
!pip install tensorflow-datasets

  Using cached tensorflow_datasets-4.6.0-py3-none-any.whl (4.3 MB)
  Using cached tensorflow_metadata-1.10.0-py3-none-any.whl (50 kB)
  Using cached promise-2.3.tar.gz (19 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached importlib_resources-5.9.0-py3-none-any.whl (33 kB)
  Using cached etils-0.8.0-py3-none-any.whl (127 kB)
  Using cached googleapis_common_protos-1.56.4-py2.py3-none-any.whl (211 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21500 sha256=3c5653fac8948e6fa2de5618e10b65671ad82dfd834aa7577e85c6f1b12107b9
  Stored in directory: c:\users\1556 mx\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise


In [2]:
!pip install keras-nlp

  Using cached keras_nlp-0.3.0-py3-none-any.whl (142 kB)
  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)


In [3]:
!pip install langdetect

  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=e41346cf404c7e84696d404d05ea9cbe50e4c1cb219842a6f3cbcaeefa6bad9a
  Stored in directory: c:\users\1556 mx\appdata\local\pip\cache\wheels\13\c7\b0\79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


## Importando as Bibliotecas

In [4]:
import pandas as pd

from langdetect import detect
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers
import re
import string

import keras_nlp
import random
import os

from tensorflow import keras
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

keras.utils.set_random_seed(42)

## Explorando os dados

In [14]:

df = pd.read_json('allData.json')
df.head(10)

,parent,caused_by,watch_list,upon_reject,sys_updated_on,x_lomei_logmein_re_logmein_session_counter,approval_history,skills,number,state,...,severity,comments,approval,due_date,sys_mod_count,reopen_count,sys_tags,location,category,u_issue_type
0,,,,cancel,2021-08-13 15:42:12,,,,INC7590026,1,...,3,,not requested,,0,0,,,software,data_issue
1,,,,cancel,2021-09-15 13:26:22,,,,INC7763698,1,...,3,,not requested,,2,0,,,software,configuration
2,,,,cancel,2021-09-13 16:54:48,,,,INC7707771,1,...,3,,not requested,,3,0,,,software,error_message_abend
3,,,,cancel,2021-09-03 08:43:31,,,,INC7723783,1,...,3,,not requested,,0,0,,,software,data_issue
4,,,,cancel,2021-08-31 17:39:25,,,,INC7707759,1,...,3,,not requested,,0,0,,,software,performance_impairment
5,,,,cancel,2021-09-08 22:28:31,,,,INC7746724,1,...,3,,not requested,,0,0,,,software,error_message_abend
6,,,,cancel,2021-08-13 13:24:02,,,,INC7585784,2,...,3,,not requested,,4,0,,,software,data_issue
7,,,,cancel,2021-09-01 18:11:36,,,,INC7255543,2,...,3,,not requested,,5,0,,,business service,data_issue
8,,,,cancel,2021-09-13 19:48:02,,,,INC7576021,2,...,3,,not requested,,23,0,,,software,data_issue
9,,,,cancel,2021-09-15 06:20:12,,,,INC7772548,2,...,3,,not requested,,2,0,,,software,data_issue


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2631 entries, 0 to 2630
Data columns (total 97 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   parent                                      2631 non-null   object        
 1   caused_by                                   2631 non-null   object        
 2   watch_list                                  2631 non-null   object        
 3   upon_reject                                 2631 non-null   object        
 4   sys_updated_on                              2631 non-null   object        
 5   x_lomei_logmein_re_logmein_session_counter  2631 non-null   object        
 6   approval_history                            2631 non-null   object        
 7   skills                                      2631 non-null   object        
 8   number                                      2631 non-null   object        
 9   state   

Como temos 97 colunas, iremos filtrar o dataframe para encontrar as colunas mais relevantes.
Primeiramente iremos filtrar apenas as colunas em string, retirando quado a coluna não possui dimensão.
Analisando, as colunas mais relevantes foram:

short_description

description

u_issue_type



## Filtrando apenas colunas em string

In [16]:
idx = df.applymap(lambda x: isinstance(x, str)).all(0)
df_new = df[df.columns[idx]]
df_new.head()

,parent,caused_by,watch_list,upon_reject,sys_updated_on,x_lomei_logmein_re_logmein_session_counter,approval_history,skills,number,sys_created_by,...,problem_id,company,activity_due,comments,approval,due_date,sys_tags,location,category,u_issue_type
0,,,,cancel,2021-08-13 15:42:12,,,,INC7590026,FR60724,...,,,,,not requested,,,,software,data_issue
1,,,,cancel,2021-09-15 13:26:22,,,,INC7763698,RF25464,...,,,,,not requested,,,,software,configuration
2,,,,cancel,2021-09-13 16:54:48,,,,INC7707771,FR60724,...,,,,,not requested,,,,software,error_message_abend
3,,,,cancel,2021-09-03 08:43:31,,,,INC7723783,TG36032,...,,,,,not requested,,,,software,data_issue
4,,,,cancel,2021-08-31 17:39:25,,,,INC7707759,PD05036,...,,,,,not requested,,,,software,performance_impairment


## Retirando as colunas com dimensão menor que 1

In [17]:
lista = df_new.columns.tolist()
lista1 = []
for indice, valor in enumerate(lista):
    print(f'{valor} : {df[valor][0]}')
    print(len(df_new[valor][0]))
 
    if len(df_new[valor][0]) > 1:
       lista1.append(valor)

parent : 
0
caused_by : 
0
watch_list : 
0
upon_reject : cancel
6
sys_updated_on : 2021-08-13 15:42:12
19
x_lomei_logmein_re_logmein_session_counter : 
0
approval_history : 
0
skills : 
0
number : INC7590026
10
sys_created_by : FR60724
7
knowledge : false
5
order : 
0
__status : success
7
cmdb_ci : 
0
cmdb_ci_business_app : 
0
active : true
4
work_notes_list : 
0
sys_domain_path : /
1
x_lomei_logmein_re_logmein_session_id : 
0
business_duration : 
0
group_list : 
0
approval_set : 
0
universal_request : 
0
short_description : Ocasional issues on calculation of condition types ZMCV and ZVAC pricing procedure ZNXENG
89
correlation_display : 
0
work_start : 
0
additional_assignee_list : 
0
service_offering : 
0
sys_class_name : incident
8
closed_by : 
0
follow_up : 
0
parent_incident : 
0
reopened_by : 
0
assigned_to : 
0
sla_due : 
0
comments_and_work_notes : 
0
u_first_assignment_group : SAP SD Parts
12
agile_story : 
0
upon_approval : proceed
7
correlation_id : 
0
made_sla : true
4
hold

In [18]:
df_new2 = df[lista1]
df_new2.head(2)

,upon_reject,sys_updated_on,number,sys_created_by,knowledge,__status,active,short_description,sys_class_name,u_first_assignment_group,...,sys_domain,caller_id,subcategory,assignment_group,description,sys_id,contact_type,approval,category,u_issue_type
0,cancel,2021-08-13 15:42:12,INC7590026,FR60724,false,success,true,Ocasional issues on calculation of condition t...,incident,SAP SD Parts,...,global,3d0e397e13236200450856022244b0b9,sap,8cf951f21bf898946e719608bd4bcb13,"In the pricing procedure ZNXENG, condition typ...",0e7211841bc63c900c7e0d076e4bcbf8,email,not requested,software,data_issue
1,cancel,2021-09-15 13:26:22,INC7763698,RF25464,false,success,true,Flash orders sent from CWIS into SAP are not a...,incident,SAP SD Parts,...,global,e71e3d7e13236200450856022244b0cb,sap,8cf951f21bf898946e719608bd4bcb13,Flash orders sent from CWIS into SAP are not a...,1a4f6c0a1b667490267d55392a4bcb37,chat,not requested,software,configuration


## Filtrando as features e os targets

In [19]:
df_new2['label'] = df_new2['u_issue_type'] 
df_new2["texto"] = df_new2['short_description'] +" "+ df_new2["description"]

<ipython-input-19-5c0d0bf0638d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new2['label'] = df_new2['u_issue_type']
<ipython-input-19-5c0d0bf0638d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new2["texto"] = df_new2['short_description'] +" "+ df_new2["description"]


In [20]:
df_final = df_new2[['texto', 'label']]
df_final.head()

,texto,label
0,Ocasional issues on calculation of condition t...,data_issue
1,Flash orders sent from CWIS into SAP are not a...,configuration
2,(DPE Reviewed) IJD-M: Shipments with multiple ...,error_message_abend
3,Delivery Header message ZFTA sometimes not exe...,data_issue
4,Report ZV_OPEN_ORD_RPT_01 issue in PAG Hello t...,performance_impairment


## Filtrando apenas os textos em inglês

In [21]:
from langdetect import detect

df_final = df_final[df_final['texto'].apply(detect).eq('en')]
df_final.head()

,texto,label
0,Ocasional issues on calculation of condition t...,data_issue
1,Flash orders sent from CWIS into SAP are not a...,configuration
2,(DPE Reviewed) IJD-M: Shipments with multiple ...,error_message_abend
3,Delivery Header message ZFTA sometimes not exe...,data_issue
4,Report ZV_OPEN_ORD_RPT_01 issue in PAG Hello t...,performance_impairment


In [22]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2532 entries, 0 to 2630
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   2532 non-null   object
 1   label   2532 non-null   object
dtypes: object(2)
memory usage: 59.3+ KB


## Como temos targets com significados iguais, iremos normalizar transformando as strings em letras minúsculas e retirando / e _

In [23]:
df_final['label'].unique()

array(['data_issue', 'configuration', 'error_message_abend',
       'performance_impairment', 'end_user', 'Connectivity',
       'failure_down', 'Error Message/Abend', 'connectivity',
       'security_threat', 'account_lockout',
       'damaged_hardware_non_warranty'], dtype=object)

In [24]:
df_final['label'] = df_final['label'].str.lower()
df_final['label'] = df_final['label'].replace("_", " ", regex=True)
df_final['label'] = df_final['label'].replace("/", " ", regex=True)
df_final['label'].unique()

array(['data issue', 'configuration', 'error message abend',
       'performance impairment', 'end user', 'connectivity',
       'failure down', 'security threat', 'account lockout',
       'damaged hardware non warranty'], dtype=object)

## Preparando o texto

In [25]:
BATCH_SIZE = 64
EPOCHS = 20
MAX_SEQUENCE_LENGTH = 512 # Somente considerar as 512 primeiras palavras de cada review
VOCAB_SIZE = 15000 # Somente considerar 15000 palavras
EMBED_DIM = 128
INTERMEDIATE_DIM = 32

In [26]:
X = df_final['texto'].values
y = df_final['label'].values

In [27]:
y = pd.get_dummies(y)
y.head(10)

,account lockout,configuration,connectivity,damaged hardware non warranty,data issue,end user,error message abend,failure down,performance impairment,security threat
0,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,1,0,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,1,0,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Testing set size: {len(test_dataset)}")

Training set size: 1417
Validation set size: 355
Testing set size: 760


## Retirando as striped_html, tranformando em letras minúsculas e retirando as pontuações

In [29]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    stripped_html1 = tf.strings.regex_replace(stripped_html, "\t", " ")
    stripped_html2 = tf.strings.regex_replace(stripped_html1, "\n", " ")
    stripped_html3 = tf.strings.regex_replace(stripped_html2, "\r", " ")
    return tf.strings.regex_replace(
        stripped_html3, f"[{re.escape(string.punctuation)}]", ""
    )

vectorizer = layers.TextVectorization(3000, standardize=custom_standardization, output_sequence_length=150)

# Para um conjunto de dados muito grande, o adapt pode remover cópias sobressalentes do conjunto de dados da memória.
vectorizer.adapt(train_dataset.map(lambda x, y: x, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE))

def vectorize_text(text, label):
    text = vectorizer(text)
    return text, label

# Vetorização dos dados e pré-busca/buffer assíncrona dos dados para melhor desempenho na GPU.
train_ds = train_dataset.batch(BATCH_SIZE).map(vectorize_text, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_dataset.batch(BATCH_SIZE).map(vectorize_text, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_dataset.batch(BATCH_SIZE).map(vectorize_text, num_parallel_calls=tf.data.AUTOTUNE)

## Criação do modelo 1 com LSTM

In [40]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(input_dim=MAX_SEQUENCE_LENGTH, output_dim=EMBED_DIM)(inputs)
x = layers.Bidirectional(layers.LSTM(INTERMEDIATE_DIM, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(INTERMEDIATE_DIM, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(INTERMEDIATE_DIM, return_sequences=True))(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(df_final['label'].unique()), activation="softmax")(x)
lstm_model = keras.Model(inputs, outputs)
lstm_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 128)         65536     
                                                                 
 bidirectional_3 (Bidirectio  (None, None, 64)         41216     
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 64)         24832     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, None, 64)         24832     
 nal)                                                            
                                                           

In [41]:
lstm_model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=[
            'accuracy',
        ],
    )

In [42]:
lstm_model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
)

Epoch 1/50
23/23 [==============================] - 15s 204ms/step - loss: 1.7655 - accuracy: 0.4665 - val_loss: 1.3755 - val_accuracy: 0.5352
Epoch 2/50
23/23 [==============================] - 1s 65ms/step - loss: 1.2661 - accuracy: 0.5702 - val_loss: 1.0332 - val_accuracy: 0.6282
Epoch 3/50
23/23 [==============================] - 2s 69ms/step - loss: 0.8974 - accuracy: 0.6951 - val_loss: 0.8492 - val_accuracy: 0.7831
Epoch 4/50
23/23 [==============================] - 2s 76ms/step - loss: 0.7370 - accuracy: 0.7918 - val_loss: 0.6977 - val_accuracy: 0.8113
Epoch 5/50
23/23 [==============================] - 2s 69ms/step - loss: 0.6484 - accuracy: 0.8158 - val_loss: 0.6930 - val_accuracy: 0.8169
Epoch 6/50
23/23 [==============================] - 2s 74ms/step - loss: 0.6180 - accuracy: 0.8207 - val_loss: 0.6928 - val_accuracy: 0.8113
Epoch 7/50
23/23 [==============================] - 2s 69ms/step - loss: 0.6279 - accuracy: 0.8179 - val_loss: 0.6926 - val_accuracy: 0.8113
Epoch 8/50


In [43]:
lstm_model.evaluate(test_ds, verbose=1, return_dict=True)

12/12 [==============================] - 1s 41ms/step - loss: 0.5782 - accuracy: 0.8355


{'loss': 0.5782259702682495, 'accuracy': 0.8355262875556946}

## Criação de modelo2 com Transformer

In [36]:
NUM_HEADS = 50
inputs = keras.Input(shape=(None,), dtype="int64", name="input_ids")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(inputs)

x = keras_nlp.layers.TransformerEncoder(intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS)(x)
x = keras_nlp.layers.TransformerEncoder(intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS)(x)
x = keras_nlp.layers.TransformerEncoder(intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS)(x)
#x = keras_nlp.layers.TransformerEncoder(intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS)(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(df_final['label'].unique()), activation="softmax")(x)

transformer_model = keras.Model(inputs, outputs)
transformer_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 128)        1985536   
 g_1 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_encoder_4 (Tran  (None, None, 128)        60492     
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_5 (Tran  (None, None, 128)        60492     
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_6 (Tran  (None, None, 128)        6049

In [37]:
transformer_model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)


In [38]:
transformer_model.fit(train_ds, epochs=20, validation_data=val_ds)


Epoch 1/20
23/23 [==============================] - 16s 504ms/step - loss: 1.2939 - accuracy: 0.5752 - val_loss: 0.7622 - val_accuracy: 0.7718
Epoch 2/20
23/23 [==============================] - 15s 663ms/step - loss: 0.7058 - accuracy: 0.7925 - val_loss: 0.6438 - val_accuracy: 0.8169
Epoch 3/20
23/23 [==============================] - 12s 506ms/step - loss: 0.5464 - accuracy: 0.8200 - val_loss: 0.6069 - val_accuracy: 0.8254
Epoch 4/20
23/23 [==============================] - 10s 446ms/step - loss: 0.4500 - accuracy: 0.8525 - val_loss: 0.5904 - val_accuracy: 0.8225
Epoch 5/20
23/23 [==============================] - 11s 494ms/step - loss: 0.3945 - accuracy: 0.8800 - val_loss: 0.7473 - val_accuracy: 0.8113
Epoch 6/20
23/23 [==============================] - 10s 422ms/step - loss: 0.3203 - accuracy: 0.9033 - val_loss: 0.6328 - val_accuracy: 0.8338
Epoch 7/20
23/23 [==============================] - 12s 511ms/step - loss: 0.1804 - accuracy: 0.9492 - val_loss: 0.9511 - val_accuracy: 0.8141

In [39]:
transformer_model.evaluate(test_ds, verbose=1, return_dict=True)

12/12 [==============================] - 2s 144ms/step - loss: 0.8738 - accuracy: 0.8368


{'loss': 0.8738236427307129, 'accuracy': 0.8368421196937561}

## Conclusão:

Utilizando o LSTM e o Transformer, a acurácia dos dados de teste foi em torno de 80%, sendo mais vantajoso a utilização do LSTM devido ao tempo no processamento dos dados de treino.